In [1]:
!pip install tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn

import tiktoken
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
text = "Hello, world. This, is a test."
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [ ]:
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result = [item.strip() for item in result if item.strip()]
print(len(result))
print(len(np.unique(result)))

4690
1130


In [ ]:
vocab = np.sort(np.unique([word for word in result]))
word_2_idx = {w:v for v,w in enumerate(vocab)}

In [ ]:
class Tokenizer_V1:
  def __init__(self, vocab: list):
    self.word2idx = {w:v for v,w in enumerate(vocab)}
    self.word2idx["<unk>"] = len(vocab) + 1
    self.word2idx["<eot>"] = self.word2idx["<unk>"] +1
    self.idx2word = {v:k for k,v in self.word2idx.items()}

  def encode(self, text: str):
    result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    result = [item.strip() for item in result if item.strip()]
    return [self.word2idx[token] if token in self.word2idx else self.word2idx["<unk>"] for token in result] + [self.word2idx["<eot>"]]

  def decode(self, ids: list):
    text = " ".join([self.idx2word[id] for id in ids])
    text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
    return text

In [ ]:
text = "I am a loser."
tokenizer = Tokenizer_V1(vocab)
ids = tokenizer.encode(text)
words = tokenizer.decode(ids)
print(text)
print(ids)
print(words)

I am a loser.
[53, 150, 115, 1131, 7, 1132]
I am a <unk>. <eot>


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
def create_data_chunks(encoded: list, context_length: int):
  inputs = []
  targets = []
  num_chunks = (len(encoded) - 1) // context_length
  for i in range(num_chunks):
    input, target = encoded[i*context_length: (i+1)*context_length], encoded[i*context_length + 1: (i+1)*context_length + 1]
    inputs.append(input)
    targets.append(target)

  return torch.tensor(inputs), torch.tensor(targets)

In [ ]:
X, Y = create_data_chunks(enc_text, 12)
print(X.shape)
print(Y.shape)
print(X[0])
print(Y[0])

torch.Size([428, 12])
torch.Size([428, 12])
tensor([   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,
          257,  7026])
tensor([  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,
         7026, 15632])


In [ ]:
class GPTDataset(torch.utils.data.Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids  = []

    token_ids = tokenizer.encode(text)
    for i in range(0, len(token_ids) - max_length, stride):
      inputs, outputs = token_ids[i: i+max_length], token_ids[i+1: i+max_length+1]
      self.input_ids.append(torch.tensor(inputs))
      self.target_ids.append(torch.tensor(outputs))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader(text, tokenizer, max_length, stride, batch_size, shuffle=True, drop_last=True):
  dataset = GPTDataset(text, tokenizer,  max_length, stride)
  return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

In [ ]:
dataloader = create_dataloader(raw_text, tokenizer, batch_size=4, max_length=32, stride=1, shuffle=False)

In [ ]:
x,y = next(iter(dataloader))
print(x.shape)

torch.Size([4, 32])


In [ ]:
emb_layer = torch.nn.Embedding(num_embeddings=tokenizer.n_vocab, embedding_dim=16)

In [ ]:
out = emb_layer(x)
print(out.shape)

torch.Size([4, 32, 16])


In [ ]:
%%writefile data_setup.py
import torch
import tiktoken


class GPTDataset(torch.utils.data.Dataset):
  def __init__(self,
               txt: str,
               tokenizer: tiktoken.Encoding,
               max_length: int,
               stride: int):
    self.input_ids = []
    self.target_ids = []
    token_ids = tokenizer.encode(txt)

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i: i+max_length]
      target_chunk = token_ids[i+1: i+1+max_length]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]


def create_dataloader(txt, tokenizer, max_length=256, stride=128, batch_size=4, shuffle=True, drop_last=True, num_workers=0):
  dataset = GPTDataset(txt, tokenizer, max_length, stride)
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
  return dataloader



Writing data_setup.py


In [ ]:
context_length = 32
embedding_dim = 16

emb_layer = torch.nn.Embedding(num_embeddings=tokenizer.n_vocab, embedding_dim=embedding_dim)
pos_encoding_layer = torch.nn.Embedding(context_length, embedding_dim)
pos_enc = pos_encoding_layer(torch.arange(context_length))

out = emb_layer(x) + pos_enc
out.shape

torch.Size([4, 32, 16])

In [3]:
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)

In [4]:
inputs.shape

torch.Size([6, 3])

In [5]:
class AttentionV1(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, inputs):
    d = torch.tensor(inputs.shape[-1])
    Q, K, V = inputs, inputs, inputs
    attention_scores = Q @ K.transpose(-1, -2)/torch.sqrt(d)
    attention_weights = torch.softmax(attention_scores, dim=-1)
    return attention_weights @ V

In [6]:
class AttentionV2(nn.Module):
  def __init__(self, d: int, qkv_bias: bool=False):
    super().__init__()
    self.d = torch.tensor(d)
    self.Wq = nn.Linear(d, d, bias=qkv_bias)
    self.Wk = nn.Linear(d, d, bias=qkv_bias)
    self.Wv = nn.Linear(d, d, bias=qkv_bias)

  def forward(self, inputs):
    Q = self.Wq(inputs)
    K = self.Wk(inputs)
    V = self.Wv(inputs)
    attention_scores = Q @ K.transpose(-1, -2)/torch.sqrt(self.d)
    attention_weights = torch.softmax(attention_scores, dim=-1)
    return attention_weights @ V

In [7]:
class CausalSelfAttention(nn.Module):
  def __init__(self,
               context_length: int,
               d: int,
               dropout_rate: float=0.1,
               qkv_bias: bool=False) -> None:
    super().__init__()
    self.d = torch.tensor(d)
    self.Wq = nn.Linear(d, d, bias=qkv_bias)
    self.Wk = nn.Linear(d, d, bias=qkv_bias)
    self.Wv = nn.Linear(d, d, bias=qkv_bias)
    self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
    self.dropout = nn.Dropout(dropout_rate)

  def forward(self, inputs):
    num_tokens = inputs.shape[-2]
    Q = self.Wq(inputs)
    K = self.Wk(inputs)
    V = self.Wv(inputs)
    attention_scores = Q @ K.transpose(-1, -2)/torch.sqrt(self.d)
    attention_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) #to account for shorter sequences
    attention_weights = torch.softmax(attention_scores, dim=-1)
    attention_weights = self.dropout(attention_weights)
    print(attention_weights)
    return attention_weights @ V



In [32]:
class MultiHeadAtttention(nn.Module):
  def __init__(self,
               context_length: int,
               embed_dim: int,
               num_heads: int,
               dropout_rate: float,
               qkv_bias=False):
    assert embed_dim % num_heads == 0
    super().__init__()
    self.d = torch.tensor(embed_dim)
    self.Wq = nn.Linear(embed_dim, embed_dim, bias=qkv_bias)
    self.Wk = nn.Linear(embed_dim, embed_dim, bias=qkv_bias)
    self.Wv = nn.Linear(embed_dim, embed_dim, bias=qkv_bias)
    self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
    self.dropout = nn.Dropout(dropout_rate)
    self.dh = int(embed_dim / num_heads)
    self.num_heads = num_heads
    self.Wo = nn.Linear(embed_dim, embed_dim, bias=qkv_bias)

  def forward(self, inputs):
    batch_size, num_tokens, embed_dim = inputs.shape
    Q = self.Wq(inputs) #shape: [B, C, dm]
    K = self.Wk(inputs) #shape: [B, C, dm]
    V = self.Wv(inputs) #shape: [B, C, dm]
    Q = Q.reshape(Q.shape[0], Q.shape[1], self.num_heads, self.dh).transpose(-2, -3) #shape: [B, nh, C, dk]
    K = K.reshape(K.shape[0], K.shape[1], self.num_heads, self.dh).transpose(-2, -3) #shape: [B, nh, C, dk]
    V = V.reshape(V.shape[0], V.shape[1], self.num_heads, self.dh).transpose(-2, -3) #shape: [B, nh, C, dk]
    attention_scores = Q @ K.transpose(-1, -2) / (self.dh)**0.5
    attention_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
    attention_weights = torch.softmax(attention_scores, dim=-1)
    attention_weights = self.dropout(attention_weights)
    context_matrix = attention_weights @ V
    context_matrix = context_matrix.transpose(-2, -3).reshape(context_matrix.shape[0], context_matrix.shape[2], self.d)
    return self.Wo(context_matrix)

In [33]:
l = MultiHeadAtttention(context_length=16, embed_dim=32, num_heads=4, dropout_rate=0.5)
input = torch.rand(1, 16, 32)
out = l(input)
print(input.shape)
print(out.shape)

torch.Size([1, 16, 32])
torch.Size([1, 16, 32])
